In [2]:
# rewrite 'comp_deconv.m'
import numpy as np
import pandas as pd
import os
from scipy.special import erfc
from scipy.interpolate import CubicSpline

# set output file
savedir = './images/comp_deconv/'
os.makedirs(savedir, exist_ok=True)

# read file
IMPR = pd.read_excel('./data/TES_detector_response_function_Fe_num0079_200804.xlsx', header=None).to_numpy()
IMPR[:, 1] = IMPR[:, 1] * 200
TES_2D = pd.read_excel('./data/CeO2_TES_2D_XES_4700_4900eV_190710_220211_cor.xlsx', sheet_name='2D_XES_TES_spectrum_XY_conv', header=None).to_numpy()
TES_2D = TES_2D.T
TES_in = pd.read_excel('./data/CeO2_TES_2D_XES_4700_4900eV_190710_220211_cor.xlsx', sheet_name='incident_x-ray_energy_eV', header=None).to_numpy()
TES_em = pd.read_excel('./data/CeO2_TES_2D_XES_4700_4900eV_190710_220211_cor.xlsx', sheet_name='emission_X-ray_energy_eV', header=None).to_numpy()

XES_2D = pd.read_excel('./data/CeO2_Analyzer_BL39XU_2D_XES.xlsx', sheet_name='2D_XES_analyzer_BL39XU', header=None).to_numpy()
XES_2D = XES_2D.T
XES_in = pd.read_excel('./data/CeO2_Analyzer_BL39XU_2D_XES.xlsx', sheet_name='incident_X-ray_energy_analyzer', header=None).to_numpy()[::-1]
XES_em = pd.read_excel('./data/CeO2_Analyzer_BL39XU_2D_XES.xlsx', sheet_name='emission_X-ray_energy_analyzer', header=None).to_numpy()

# initialize
ana_x_array = XES_em.flatten()
ana_y_array = XES_in.flatten()
d_y = np.diff(ana_y_array, prepend=ana_y_array[0], append=ana_y_array[-1])

tes_x_array = np.arange(XES_em[0, 0], XES_em[-1, 0] + 1)
tes_y_array = XES_in.flatten()

sz_x = len(XES_em)
sz_y = len(XES_in)
NN = len(tes_x_array)
LL = len(tes_y_array)

G_i = np.zeros((sz_y, LL))

# ガウス関数の畳み込み行列
sigma = 2 / 2.35  # 半値幅2eV
G_i[0, :] = erfc(-(tes_y_array - ana_y_array[0]) / (np.sqrt(2) * sigma)) - erfc(
    -(tes_y_array - ana_y_array[0] - d_y[0] / 2) / (np.sqrt(2) * sigma)
)
for k in range(1, sz_y):
    G_i[k, :] = erfc(-(tes_y_array - ana_y_array[k] + d_y[k - 1] / 2) / (np.sqrt(2) * sigma)) - erfc(
        -(tes_y_array - ana_y_array[k] - d_y[k] / 2) / (np.sqrt(2) * sigma)
    )

G_f = np.zeros((sz_x, NN))

# 装置関数の畳み込み行列
for k in range(sz_x):
    pt = ana_x_array[k]
    cs = CubicSpline(IMPR[:, 0], IMPR[:, 1])
    G_f[k, :] = np.maximum(cs(tes_x_array - pt), 0)

# flattern
XES_2D_vec = XES_2D.flatten()
K = np.kron(G_f.T, G_i.T)
Y_vec = np.dot(K, XES_2D_vec)
YY = Y_vec.reshape((LL, NN))

# ノイズ付加
YY_noise = YY + np.sqrt(np.abs(YY)) * np.random.randn(*YY.shape)
np.savetxt(os.path.join(savedir, "conv_noise_2eV.csv"), YY_noise, delimiter=',')


In [3]:
print(f"length of XES_em: {sz_x}")
print(f"length of XES_in: {sz_y}")
print(f"length of tes_x_array: {NN}")
print(f"length of tes_y_array: {LL}")

length of XES_em: 226
length of XES_in: 73
length of tes_x_array: 46
length of tes_y_array: 73


In [5]:
G_i

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.22843348e-09, 9.96691298e-01, 3.30869749e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.22385385e-18, 3.30870172e-03, 1.99338260e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.99338260e+00, 3.30870172e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.30870172e-03, 1.99338260e+00, 3.30870172e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.22385385e-18, 3.30870172e-03, 1.99338260e+00]])